In [1]:
import pandas as pd
import random
import numpy as np
from joblib import load
import glob
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import random
import sklearn.model_selection as ms
from sklearn import ensemble

In [2]:
path = '../data/weatherdata'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

weather_df = pd.concat(li, axis=0, ignore_index=True)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (43,44,48,53,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (45,46,50,55,59,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,56,57,64,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,49,54,58,59) have mi

In [3]:
def to_milesperhour(num):
    """A function to convert wind speed from meters/second to miles/hour. The conversion is done by using the 
    above formula
    """
    convert = (num * 3600)/(1000 * 1.6)
    return convert

In [4]:
def clean_weatherdata(df):
    """This function will clean the weather data from any given year or years (the merged weather data)
    Weather data was obtained from NOAA( National Oceanic and Atmospheric Administration ) 
    https://www.ncei.noaa.gov/data/global-hourly/archive/csv/.
    Additional websites were also used as references in order to interpret the numbers from the data and to
    engineer new features such as windy and rainy. The column windy and its values (breeze, gale, force...)
    were all based on information from NOAA https://www.weather.gov/pqr/wind. 
    https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/
    was used to interpret the numbers of the data.
    """
    # Make a copy
    nyweather= df.copy()
    
    # Select the columns that will be used
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',
             'DEW', 'SLP', 'AA1','AA2']]
    
    # Clean DATE column
    nyweather['DATE'] = nyweather['DATE'].str.replace('T', ' ')
    
    # Clean TMP (temperature) column. Please refer to 
    # Interpreting the numbers are based on 
    #https://www.visualcrossing.com/resources/documentation/weather-data/how-we-process-integrated-surface-database-historical-weather-data/

    nyweather['TMP'] = nyweather['TMP'].astype(str)
    nyweather['TMP'] = nyweather['TMP'].str.slice(0, -2)
    nyweather['TMP'] = nyweather['TMP'].str.replace('+','')
    nyweather = nyweather.drop(nyweather[nyweather.TMP == '9999'].index)
    nyweather['TMP'] = nyweather['TMP'].astype(int)
    nyweather['TMP'] = nyweather['TMP']/10
    print('complete part 1')
    
    # Clean WND column
    # Please refer to https://www.weather.gov/pqr/wind
    nyweather['WND'] = nyweather['WND'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.WND.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={3:'wind_speed'})
    nyweather['wind_speed'] = nyweather['wind_speed'].astype(int)
    nyweather['wind_speed'] = nyweather['wind_speed']/10
    nyweather['wind_speed'] = nyweather['wind_speed'].map(to_milesperhour)
    
    # Engineered a new feature called windy
    nyweather['windy'] = nyweather['wind_speed'].apply(lambda x: 'calm' if x < 4 else 'breeze' if x < 12 else \
                                        'moderate breeze' if x < 24 else 'strong breeze' if x < 31 else \
                                        'gale' if x < 63 else 'storm force')
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'AA1','AA2', 'wind_speed', 'windy']]
    
    print('complete part 2')
    
    # Clean the AA1 column that has precipitation information
    nyweather['AA1'] = nyweather['AA1'].fillna('0,0,0,0')
    nyweather['AA1'] = nyweather['AA1'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.AA1.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={1:'precipitation'})
    nyweather['precipitation'] = nyweather['precipitation'].astype(int)
    nyweather['precipitation'] = nyweather['precipitation']/10
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',\
                           'DEW', 'SLP', 'wind_speed', 'windy', 'precipitation']]
    
    # Engineered a new feature called rainy
    nyweather['rainy'] = nyweather['precipitation'].apply(lambda x: 'rainy' if x > 0 else 'not rainy')
    
    
    # Clean DATE column
    # Generate new features - month, date, hour and month2
    nyweather['DATE'] = nyweather['DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    nyweather['month'] = nyweather.DATE.dt.month
    nyweather['date'] = nyweather.DATE.dt.date
    nyweather['hour'] = nyweather.DATE.dt.hour
    nyweather['month2'] = nyweather.DATE.dt.month.apply(lambda x: 'Jan' if x == 1 else 'Feb' if x == 2 else \
    'Mar' if x == 3 else 'Apr' if x == 4 else 'May' if x == 5 else 'Jun' if x == 6 else 'Jul' if x == 7 else \
    'Aug' if x == 8 else 'Sep' if x == 9 else 'Oct' if x == 10 else 'Nov' if x == 11 else 'Dec' )
    
    # Generate cleaned dataframe
    return nyweather

In [5]:
weather_df = clean_weatherdata(weather_df)

complete part 1
complete part 2


In [6]:
daily_temp = weather_df.groupby(["date", "hour"]).agg({
  "TMP" : ['min', 'max', 'mean'], 
  "wind_speed" : ["min", 'max', 'mean'], 
  "precipitation" : ["sum"]
  }).reset_index()
daily_temp.columns = ["_".join(x) for x in daily_temp.columns.ravel()]
daily_temp = daily_temp.rename(columns = {'date_': 'date', 'hour_':'hour'})

In [7]:
def downsampling_out():
    sampled_list = []
    n = 0
    while n <= 3:
        print(n)
        p = 0.01  # 1% of the lines
        # keep the header, then take only 1% of lines
        # if random from [0,1] interval is greater than 0.01 the row will be skipped
        df = pd.read_csv('../data/outgoing.csv.gz', skiprows=lambda i: i>0 and random.random() > p)
        df['Sample_num'] = n
        sampled_list.append(df)
        n += 1
    return pd.concat(sampled_list)

In [8]:
outgoing_sample = downsampling_out()

0
1
2
3


In [9]:
def downsampling_in():
    sampled_list = []
    n = 0
    while n <= 3:
        print(n)
        p = 0.01  # 1% of the lines
        # keep the header, then take only 1% of lines
        # if random from [0,1] interval is greater than 0.01 the row will be skipped
        df = pd.read_csv('../data/incoming.csv.gz', skiprows=lambda i: i>0 and random.random() > p)
        df['Sample_num'] = n
        sampled_list.append(df)
        n += 1
    return pd.concat(sampled_list)

In [10]:
incoming_sample = downsampling_in()

0
1
2
3


In [11]:
#outgoing = pd.read_csv('../data/outgoing.csv.gz')

In [12]:
#incoming = pd.read_csv('../data/incoming.csv.gz')

In [13]:
outgoing=outgoing_sample.copy()
incoming=incoming_sample.copy()

In [14]:
outgoing['date'] = pd.to_datetime(outgoing['start_date'])

In [15]:
incoming['date'] = pd.to_datetime(incoming['stop_date'])

In [16]:
# Feature Engineering
outgoing['month'] = outgoing.date.dt.month
outgoing['dayofweek2'] = outgoing['date'].dt.weekday
outgoing['season2'] = outgoing['season'].apply(lambda x: 1 if x == 'spring' else 2 if x == 'summer' else \
                                             3 if x == 'fall' else 4)

In [17]:
# Feature Engineering
incoming['month'] = incoming.date.dt.month
incoming['dayofweek2'] = incoming['date'].dt.weekday
incoming['season2'] = incoming['season'].apply(lambda x: 1 if x == 'spring' else 2 if x == 'summer' else \
                                             3 if x == 'fall' else 4)

In [18]:
outgoing = outgoing[['start station id', 'start_date', 'date', 'month', 'dayofweek2', 'season2', 'start_hour',\
                    'outgoing_bike_count', 'bike_demand']].rename(columns = {'start_hour': 'hour'})

In [19]:
incoming = incoming[['end station id', 'stop_date', 'date', 'month', 'dayofweek2', 'season2', 'stop_hour',\
                    'incoming_bike_count', 'dock_demand']].rename(columns = {'stop_hour': 'hour'})

In [20]:
daily_temp['date'] = pd.to_datetime(daily_temp['date'])

In [21]:
combined_outgoing = outgoing.merge(daily_temp, on = ['date','hour'], how = 'inner')

In [22]:
combined_incoming = incoming.merge(daily_temp, on = ['date','hour'], how = 'inner')

In [23]:
combined_outgoing

,start station id,start_date,date,month,dayofweek2,season2,hour,outgoing_bike_count,bike_demand,TMP_min,TMP_max,TMP_mean,wind_speed_min,wind_speed_max,wind_speed_mean,precipitation_sum
0,72,2013-07-07,2013-07-07,7,6,2,10,2,Medium,27.8,27.8,27.8,10.350,10.35,10.3500,0.0
1,146,2013-07-07,2013-07-07,7,6,2,10,3,Medium,27.8,27.8,27.8,10.350,10.35,10.3500,0.0
2,329,2013-07-07,2013-07-07,7,6,2,10,1,Low,27.8,27.8,27.8,10.350,10.35,10.3500,0.0
3,398,2013-07-07,2013-07-07,7,6,2,10,1,Low,27.8,27.8,27.8,10.350,10.35,10.3500,0.0
4,322,2013-07-07,2013-07-07,7,6,2,10,1,Low,27.8,27.8,27.8,10.350,10.35,10.3500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891164,3781,2020-02-05,2020-02-05,2,2,4,1,1,Low,13.9,13.9,13.9,0.000,0.00,0.0000,0.3
891165,3812,2020-02-09,2020-02-09,2,6,4,4,1,Low,0.6,0.6,0.6,8.100,8.10,8.1000,0.0
891166,3830,2019-12-21,2019-12-21,12,5,4,5,1,Low,-3.9,-3.9,-3.9,8.100,8.10,8.1000,0.0
891167,3842,2020-04-08,2020-04-08,4,2,1,1,1,Low,12.8,12.8,12.8,8.100,8.10,8.1000,0.0


In [41]:
X = combined_outgoing[['start station id', 'hour','dayofweek2', 'TMP_mean']]

In [42]:
y = combined_outgoing['bike_demand']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 80% training and 20% test

In [44]:
clfout=RandomForestClassifier(bootstrap=True, n_estimators=100, max_depth = 22, criterion = 'gini', class_weight = 'balanced')

In [45]:
clfout.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=22)

In [46]:
y_pred=clfout.predict(X_test)

In [47]:
y_pred

array(['High', 'Low', 'Low', ..., 'Low', 'High', 'Medium'], dtype=object)

In [30]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5710358293030511


In [31]:
import pickle

In [48]:
Pkl_Filename = "clf_out.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clfout, file)

In [33]:
X = combined_incoming[['end station id', 'hour', 'dayofweek2', 'TMP_mean']]

In [34]:
y = combined_incoming['dock_demand']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 80% training and 20% test

In [36]:
clfin=RandomForestClassifier(bootstrap=True, n_estimators=100, max_depth = 22, criterion = 'gini', class_weight = 'balanced')

In [37]:
clfin.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=22)

In [38]:
y_pred=clfin.predict(X_test)

In [39]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.577696025818486


In [40]:
Pkl_Filename = "clf_in.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clfin, file)